# Анализ данных с помощью SQL

* **Запросы расположены в порядке возрастания сложности (от простого к более сложным)**
* **Cоздание и заполнение таблиц данными представлено в отдельном файле**
* **Схема БД представлена в файле "Readme.md"**


### 1. Для каждого сотрудника определить, какой процент по отношению к зарплате составляют его комиссионные.


In [ ]:
SELECT last_name,
       Round(Coalesce(commission / salary, 0) * 100, 2) AS SHARE
FROM   employee


### 2. Вывести два инициала (с точками) и фамилии всех сотрудников


In [ ]:
SELECT SUBSTR(first_name, 1, 1) || '.' AS I1,
       middle_initial || '.' AS I2,
       last_name
FROM employee

### 3. По каждой сделке вывести точную сумму сделки, сумму сделки, округленную в большую сторону и округленную в меньшую сторону,


In [ ]:
# Cумма сделки, округленная в большую сторону
SELECT Order_ID,
       CEILING (Total)
FROM Sales_order

,order_id,ceiling
0,610,102.0
1,611,45.0
2,612,5860.0
3,601,61.0
4,602,56.0
...,...,...
95,530,3027.0
96,545,475.0
97,557,2462.0
98,546,3663.0


In [ ]:
# Cумма сделки, округленная в меньшую сторону
SELECT Order_ID,
       FLOOR (Total)
FROM Sales_order

,order_id,floor
0,610,101.0
1,611,45.0
2,612,5860.0
3,601,60.0
4,602,56.0
...,...,...
95,530,3026.0
96,545,475.0
97,557,2461.0
98,546,3663.0


### 4. Выбрать фамилии всех сотрудников, фамилии которых начинаются на букву 'S'.


In [ ]:
SELECT  last_name
FROM employee
WHERE SUBSTR (last_name, 1, 1) = 's'

,last_name
0,smith
1,shaw
2,sommers
3,scott


### 5. Выбрать всех сотрудников, имена которых состоят из 6 букв и начинаются на 'MAR'.


In [ ]:
SELECT  first_name, last_name
FROM employee
WHERE LENGTH (first_name) = 6 and SUBSTR (first_name, 1, 3) = 'mar'

,first_name,last_name
0,marion,blake


### 6. Для каждого сотрудника выбрать количество полных лет работы в фирме.


In [ ]:
SELECT last_name, DATE_PART('day', (CURRENT_DATE - Hire_Date) / 365.25 )  AS Data
FROM employee

### 7. Выбрать количество сотрудников, получающих комиссионные. Если сотрудник получает 0 комиссионных, то это тоже считается. Не считаются только NULL.


In [ ]:
SELECT count (employee_id)
FROM   employee
WHERE  commission IS NOT NULL

,count
0,10


### 8. Выбрать средние зарплаты продавцов (код должности - 670) и клерков (код должности - 667).


In [ ]:
SELECT Job_id, avg(salary)
FROM   employee
GROUP BY  Job_ID
Having Job_ID =  670 or Job_ID =  667

### 9. Выбрать коды продуктов, по которым было совершено меньше 10 продаж.




In [ ]:
SELECT product_id, COUNT(order_id) AS count_of_sales
FROM item
GROUP BY product_id
HAVING COUNT(order_id)<10

### 10. Выбрать максимальную зарплату продавцов (код должности - 670) по каждому отделу.


In [ ]:
SELECT department_id, max(salary)
FROM   employee
WHERE Job_ID =  670
GROUP BY  department_id

,max,department_id
0,1600.0,30
1,1250.0,13
2,1300.0,43
3,1500.0,23


### 11. Выбрать всю информацию о каждом отделе и месте его расположения.


In [ ]:
SELECT
  department_id,
  name,
  department.location_id,
  location.regional_group
FROM
  department
  JOIN
  location
  ON department.location_id = location.location_id

### 12. Выбрать список фамилий сотрудников по городам, где они работают, с указанием для каждого его должности.


In [ ]:
SELECT
  last_name,
  job.function,
  location.regional_group
FROM
  employee
  JOIN
  job
  ON employee.job_id = job.job_id
  JOIN
  department
  ON employee.department_id = department.department_id
  JOIN
  location
  ON department.location_id = location.location_id

### 13. Выбрать всех покупателей, с которыми работает продавец TURNER.


In [ ]:
SELECT
  name
FROM
  customer
WHERE salesperson_id IN (SELECT employee_id FROM employee WHERE last_name = 'turner')

,name
0,jocksports
1,k + t sports
2,north woods health and fitness supply center


### 14. Выбрать сотрудников, которые не закреплены за конкретным заказчиком.


In [ ]:
SELECT
  first_name,
  last_name
FROM
  employee
  left JOIN
  customer
  ON employee.employee_id = customer.salesperson_id
WHERE customer_id is null

### 15. Выбрать имена продавцов, которые работают более чем с одним покупателем и названия покупателей, которых они обслуживают.


In [ ]:
SELECT
  first_name,
  customer.name
FROM
  employee
  JOIN
  customer
  ON employee.employee_id = customer.salesperson_id
  WHERE salesperson_id IN (SELECT salesperson_id FROM customer GROUP BY salesperson_id HAVING count(name) >1 )

### 16. Выбрать сумму всех продаж, которые обеспечил продавец TURNER.


In [ ]:
SELECT
  sum(total),
  employee.last_name
FROM
  sales_order
  JOIN
  customer
  ON sales_order.customer_id = customer.customer_id
  JOIN
  employee
  ON customer.salesperson_id = employee.employee_id
  WHERE last_name = 'turner'
  group by last_name


,sum,last_name
0,58055.9,turner


### 17. Выбрать сумму всех продаж товаров, связанных с теннисом, за лето 1990г.


In [ ]:
SELECT
  sum(item.total),
  product.description
FROM
  sales_order
  JOIN
  item
  ON sales_order.order_id = item.order_id
  JOIN
  product
  ON item.product_id = product.product_id
  WHERE product.description LIKE '%tennis%' AND order_date  BETWEEN '19900601' AND '19900831'
  GROUP BY description


,sum,description
0,1468.5,ace tennis balls-3 pack
1,16.8,ace tennis balls-6 pack
2,514.0,ace tennis net
3,608.0,ace tennis racket i
4,4624.0,ace tennis racket ii
5,37.4,"rh: ""guide to tennis"""
6,1224.0,sp tennis racket


### 18. Выбрать имена и зарплаты самого старого и самого молодого сотрудника фирмы. ("Возраст" сотрудника определяется по дате его поступления на работу в фирму).


In [ ]:
SELECT
    last_name,
    salary,
    hire_date
FROM
  employee
  WHERE hire_date = (SELECT MAX(hire_date) FROM employee ) or hire_date = (SELECT MIN(hire_date) FROM employee )


,last_name,salary,hire_date
0,smith,800.0,1984-12-17
1,murray,750.0,1987-01-16


### 19. Для каждого продавца вывести разность между его зарплатой и средней зарплатой продавцов в фирме.


In [ ]:
SELECT
    employee_id,
    job.function,
    salary - (SELECT AVG(salary) as mean_salary FROM employee WHERE function = 'salesperson' AND employee.job_id = job.job_id) as difference
FROM
  employee
    JOIN
  job
  ON employee.job_id = job.job_id
  WHERE function = 'salesperson'


,employee_id,difference
0,7499,268.181818
1,7521,-81.818182
2,7555,-81.818182
3,7557,-81.818182
4,7560,-81.818182
5,7564,-81.818182
6,7600,-81.818182
7,7654,-81.818182
8,7789,168.181818
9,7820,-31.818182


### 20. Вывести имена, средние инициалы и фамилии всех однофамильцев.


In [ ]:
SELECT first_name,
  middle_initial,
  last_name
  FROM employee
  WHERE last_name in( SELECT last_name from employee group by last_name having count(*) > 1)


,first_name,middle_initial,last_name


### 21. Для каждого отдела выбрать процент, который составляют сотрудники отдела от общего числа сотрудников фирмы, и процент, который составляет зарплата отдела от общей зарплаты фирмы.


In [ ]:
SELECT CAST(COUNT(employee_id) AS FLOAT) / CAST((SELECT COUNT(employee_id)
FROM employee) AS FLOAT) * 100 AS percent_of_employee,
CAST (sum(salary) AS FLOAT) / CAST((SELECT sum(salary)
FROM employee) AS FLOAT) * 100 AS percent_of_salary
FROM employee
GROUP BY department_id


,percent_of_employee,percent_of_salary
0,3.125,3.047776
1,3.125,2.141680
2,9.375,14.415157
3,12.500,15.939044
4,3.125,2.965404
5,18.750,15.485997
6,3.125,3.624382
7,15.625,17.915980
8,15.625,12.108731
9,15.625,12.355848


### 22. Выбрать фамилии начальников тех отделов, которые имеют разветвленную структуру (т.е. тех начальников, у прямых подчиненных которых есть свои подчиненные).

In [ ]:
SELECT
    last_name
  FROM
    employee
  WHERE employee_id in (SELECT manager_id FROM employee WHERE employee_id in (SELECT employee_id FROM employee WHERE employee_id in (SELECT manager_id FROM employee WHERE not employee_id is null) ) )


,last_name
0,jones
1,alberts
2,king


### 23. Выбрать названия отдела, заключившего сделки на самую большую сумму, и города, в котором он находится.


In [ ]:
SELECT
    department.name,
    location.regional_group
FROM
  department
  JOIN
  location
  ON department.location_id = location.location_id
  JOIN
  employee
  ON department.department_id = employee.department_id
  JOIN
  customer
  ON employee.employee_id = customer.salesperson_id
  JOIN
  sales_order
  ON customer.customer_id = sales_order.customer_id
  WHERE total = (SELECT MAX(total) FROM sales_order )


,name,regional_group
0,sales,chicago


### 24. Выбрать название товара, дату продажи, цену продажи для всех случаев, когда товары продавались ниже, чем за 75% их объявленной цены.

**Цена действует от price.start_date до price.end_date, причем если price.end_date IS NULL, то цена действует до сих пор**

In [ ]:
SELECT
    list_price,
    product.description as product_name,
    item.actual_price,
    sales_order.ship_date,
    actual_price*100/list_price AS percent_of_price
FROM
    price
  JOIN
  product
    ON price.product_id = product.product_id
  JOIN
  item
    ON product.product_id = item.product_id
  JOIN
  sales_order
    ON item.order_id = sales_order.order_id
  WHERE ((order_date BETWEEN start_date AND end_date ) or (start_date <= order_date and end_date is null)) and actual_price*100/list_price < 75


,list_price,product_name,actual_price,ship_date,percent_of_price
0,2.4,sb energy bar-6 pack,1.75,1991-02-13,72.916667
1,15.0,yellow jersey bicycle gloves,10.00,1990-09-07,66.666667
2,15.0,yellow jersey bicycle gloves,10.00,1991-03-07,66.666667


### 25. Выбрать общую сумму скидок, предоставленных покупателю STADIUM SPORTS в 1989г.

**Скидка здесь это list_price*quantity-item.total**

In [ ]:
SELECT sum(list_price*item.quantity-item.total) AS discount
FROM price
JOIN product
	ON price.product_id = product.product_id
JOIN item
	ON product.product_id = item.product_id
JOIN sales_order
	ON item.order_id = sales_order.order_id
JOIN customer
	ON sales_order.customer_id = customer.customer_id
WHERE name LIKE '%stadium sports%' AND order_date BETWEEN '19890101' AND '19891231'
AND (order_date>=start_date AND (order_date<end_date OR end_date IS NULL))


,discount
0,356.9


### 26. Выбрать список сотрудников фирмы с указанием фамилии непосредственного начальника каждого.


In [ ]:
SELECT worker.last_name AS worker,
		 manager.last_name AS manager
FROM employee worker, employee manager
WHERE worker.manager_id = manager.employee_id


### 27. Для каждого сотрудника вывести разность между его зарплатой и средней зарплатой сотрудников, выполняющих те же функции.


In [ ]:
 SELECT last_name,
		 salary - avg_sal AS avg_sl
FROM employee,
	(SELECT job_id, avg(salary) AS avg_sal FROM employee GROUP BY  job_id) avg_slry
WHERE employee.job_id = avg_slry.job_id

### 28. Вывести таблицу распределения объема продаж товара DUNK BASKETBALL INDOOR по годам (объем продаж в деньгах).


In [ ]:
SELECT SUM(item.total) as volume_of_sales,
		 EXTRACT(year FROM order_date) as year
FROM sales_order
JOIN item
	ON sales_order.order_id = item.order_id
JOIN product
	ON item.product_id = product.product_id
WHERE description = 'dunk basketball indoor'
GROUP BY year
ORDER BY  year asc


,volume_of_sales,year
0,5830.0,1989.0
1,5337.3,1990.0
2,3597.0,1991.0


### 29. Выбрать фамилии тех сотрудников, у которых суммарный доход (зарплата + комиссионные) больше 2000.


In [ ]:
SELECT last_name
FROM   employee
WHERE salary + COALESCE(commission, 0)  > 2000


,last_name
0,doyle
1,dennis
2,baker
3,shaw
4,jones
5,alberts
6,porter
7,martin
8,blake
9,clark


### 30. Выбрать названия товаров, для которых нынешняя цена увеличилась по сравнению с ценой на 15 декабря 1989г. более, чем на 15%.

In [ ]:
SELECT new.product_id,
old.list_price as old_as_price,
new.list_price as new_as_price,
new.list_price/old.list_price*100-100 as percent,
description
FROM price old
JOIN price new
ON old.product_id = new.product_id
JOIN product
ON new.product_id = product.product_id
WHERE new.end_date IS NULL AND ((old.start_date <= '1989-12-15' AND old.end_date > '1989-12-15') OR (old.start_date <= '1989-12-15' AND old.end_date IS NULL)) and new.list_price/old.list_price*100-100  > 15
ORDER BY 1


,product_id,old_as_price,new_as_price,percent,description
0,100860,30.0,35.0,16.666667,ace tennis racket i
1,100861,39.0,45.0,15.384615,ace tennis racket ii
2,104352,50.0,58.3,16.600000,dunk basketball professional
3,105124,10.0,15.0,50.000000,yellow jersey bicycle gloves
4,105125,2.0,3.0,50.000000,yellow jersey water bottle
5,105126,5.0,6.0,20.000000,yellow jersey bottle cage


### 31. Выбрать только те месяцы, сумма продаж за которые превысила среднюю за год.

In [ ]:
with avg_totals as (
  select
    to_char (order_date, 'YYYY') as year,
    avg(total) as avg_tot
  from sales_order
  group by year
  order by year
), totals AS (
select
  to_char (order_date, 'YYYY') as year_sal,
  to_char (order_date, 'YYYY-MM') as month_sal,
  total
from sales_order
)
select
  totals.month_sal as month,
  max(totals.total) as total,
  round(avg_totals.avg_tot, 2) as avg_total
from totals
  join avg_totals on avg_totals.year = totals.year_sal
where totals.total >= avg_totals.avg_tot
group by totals.month_sal, round(avg_totals.avg_tot, 2)
order by month_sal;



### 32. Вывести среднюю зарплату работников со "стажем" 37 лет

In [ ]:
with experience as (
select salary
from employee
where ((extract(year from current_timestamp)) - (extract(year from hire_date))) = 37
)
select avg(salary) from experience

### 33. Создать классификацию для товаров по степени задержки их отгрузки. Вывести те товары, которые были отгружены с задержкой. (Задержкой считается ситуация, когда между заказом и отгрузкой прошло более 10 дней).

In [ ]:
WITH delay AS (
  SELECT
    order_id,
    order_date,
	ship_date,
    (CASE
        WHEN (extract(day from ship_date)) - (extract(day from order_date)) > 10 THEN 'Отгружено с задержкой'
        WHEN (extract(day from ship_date)) - (extract(day from order_date)) <= 10 THEN 'Отгружено без задержки'
     END) as class
  FROM
    sales_order
)
SELECT
    delay.order_id,
	product.description,
    delay.order_date,
	delay.ship_date,
    delay.class
    FROM
        delay
	JOIN item
	on item.order_id = delay.order_id
	JOIN product
	ON product.product_id = item.product_id
    WHERE
        class = 'Отгружено с задержкой';

### 34. Посчитать максимальную зарплату по каждому отделу. Также посчитать пропорцию зарплат в отделе относительно суммы всех зарплат в этом отделе, а также относительно всего фонда оплаты труда. (Использовать оконные функции)

In [ ]:
SELECT
    employee.first_name,
    employee.last_name,
    department.name as department_name,
    employee.salary,
    MAX(salary) OVER (PARTITION BY department.name) as max_salary,
	  ROUND(CAST(employee.salary AS numeric(9,2)) / SUM(employee.salary) OVER (PARTITION BY department.name) * 100, 2) as dep_ratio,
    ROUND(CAST(employee.salary AS numeric(9,2)) / SUM(employee.salary) OVER () * 100, 2) as total_ratio
FROM employee
JOIN department
ON department.department_id = employee.department_id;

### 35. Вывести ранжирование зарплат по каждому отделу. (Посчитать ранги с пропуском и без).

In [ ]:
SELECT
    employee.first_name,
    employee.last_name,
    department.name as department_name,
    employee.salary,
	RANK() OVER (PARTITION BY department.name order by salary desc) as rank,
	DENSE_RANK() OVER (PARTITION BY department.name order by salary desc) as dense_rank
FROM employee
JOIN department
ON department.department_id = employee.department_id;

### 36. Вывести информацию по заказу с ID = 601, использовав хранимую функцию.

In [ ]:
CREATE function id_orders (arg int)
returns sales_order
LANGUAGE sql AS
$$
    SELECT *
    FROM sales_order
    WHERE order_id = arg;
$$ ;

select * from id_orders(601)


### 37. Добавить новое значение в таблицу Product, использовав хранимую процедуру.

In [ ]:
CREATE PROCEDURE Addprod (
	prod_id int,
	desc_prod varchar(100)
)
LANGUAGE plpgsql AS
$$
BEGIN
	INSERT INTO product (product_id, description) VALUES
	(prod_id,
	desc_prod
    );
END
$$;

CALL Addprod (200174, 'bandage');